<a href="https://colab.research.google.com/github/tyri0n11/distributed-system/blob/main/7_1_classification_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Binary Classification


# Create Spark entry points

In [3]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

In [4]:
sc = SparkContext(conf=SparkConf().setAppName("appName").setMaster("local[2]"))
spark = SparkSession(sparkContext=sc)

# Step 1: As usual, define the problem

**What sorts of people were likely to survive from the Titanic accident?**

# Step 2: gather the data

The datasets can be found here: [https://www.kaggle.com/c/titanic/data](https://www.kaggle.com/c/titanic/data).
It is also available in this github repository:
    
* [data/titanic/gender_submission.csv](data/titanic/gender_submission.csv)
* [data/titanic/test.csv](data/titanic/test.csv)
* [data/titanic/train.csv](data/titanic/train.csv)

In [5]:
!kaggle competitions download -c titanic

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 4, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.12/dist-packages/kaggle/__init__.py", line 6, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 434, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.config/kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


# Step 3: prepare data for consumption

In [6]:
# load packages
import sys
print('Python version: {}'. format(sys.version))

import pandas as pd
print('Python version: {}'. format(pd.__version__))

import matplotlib
print('matplotlib version: {}'. format(matplotlib.__version__))

import numpy as np
print('numpy version: {}'. format(np.__version__))

import scipy as sp
print('scipy version: {}'. format(sp.__version__))

import IPython
from IPython import display # pretty printing of dataframe in Jupyter notebook
print('IPython version: {}'. format(IPython.__version__))

import pyspark
print('Apache Spark Pyspark version: {}'. format(pyspark.__version__)) # pyspark version

# misc libraries
import random
import time

# ignore warnings
import warnings
warnings.filterwarnings('ignore')
print('-'*25)

Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Python version: 2.2.2
matplotlib version: 3.10.0
numpy version: 2.0.2
scipy version: 1.16.3
IPython version: 7.34.0
Apache Spark Pyspark version: 4.0.1
-------------------------


In [7]:
from pyspark.ml.classification import LinearSVC
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.classification import OneVsRest

In [8]:
from subprocess import check_output
print('-'*10, 'Files', '-'*10)
print(check_output(['ls', './']).decode('utf8'))

---------- Files ----------
gender_submission.csv
sample_data
test.csv
train.csv



In [9]:
data_raw = spark.read.csv('./train.csv', inferSchema=True, header=True)

# the test file provided is for validation of final model.
data_val = spark.read.csv('./test.csv', inferSchema=True, header=True)

# preview the data
# data type
print('-'*10, 'data types', '-'*10)
pd.DataFrame(data_raw.dtypes)

---------- data types ----------


,0,1
0,PassengerId,int
1,Survived,int
2,Pclass,int
3,Name,string
4,Sex,string
5,Age,double
6,SibSp,int
7,Parch,int
8,Ticket,string
9,Fare,double


In [10]:
# data summary
print('-'*10, 'data summary', '-'*10)
data_raw.describe().toPandas()

---------- data summary ----------


,summary,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,count,891,891,891,891,891,714,891,891,891,891,204,889
1,mean,446.0,0.3838383838383838,2.308641975308642,None,None,29.69911764705882,0.5230078563411896,0.38159371492704824,260318.54916792738,32.2042079685746,None,None
2,stddev,257.3538420152301,0.48659245426485753,0.8360712409770491,None,None,14.526497332334035,1.1027434322934315,0.8060572211299488,471609.26868834975,49.69342859718089,None,None
3,min,1,0,1,"""Andersson, Mr. August Edvard (""""Wennerstrom"""")""",female,0.42,0,0,110152,0.0,A10,C
4,max,891,1,3,"van Melkebeke, Mr. Philemon",male,80.0,8,6,WE/P 5735,512.3292,T,S


In [ ]:
# view a small subset of the data
print('-'*10, 'randomely sample 1% data to view', '-'*10)
data_raw.randomSplit([0.01, 0.99])[0].toPandas()

---------- randomely sample 1% data to view ----------


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,376,1,1,"Meyer, Mrs. Edgar Joseph (Leila Saks)",female,NaN,1,0,PC 17604,82.1708,None,C
1,458,1,1,"Kenyon, Mrs. Frederick R (Marion)",female,NaN,1,0,17464,51.8625,D21,S
2,484,1,3,"Turkula, Mrs. (Hedwig)",female,63.0,0,0,4134,9.5875,None,S
3,816,0,1,"Fry, Mr. Richard",male,NaN,0,0,112058,0.0000,B102,S


### 3.21 The 4 C's of data cleaning: Correcting, Completing, Creating, and Converting

In [ ]:
# we first check which values are NULL values for each column
# then we convert the boolean values to int (0 and 1), then we can count how many 1's exist in each column.
print('-'*25)
print('0: is not NULL')
print('1: is NULL')
print('-'*25)
print(' '*25)
# we build column strings and then use eval() to convert strings to column expressions.
data_raw.select([eval('data_raw.' + x + '.isNull().cast("int").alias("' + x + '")') for x in data_raw.columns]).show(n=10)

-------------------------
0: is not NULL
1: is NULL
-------------------------
                         
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|  0|    0|    0|     0|   0|    1|       0|
|          0|       0|     0|   0|  0|  0|    0|    0|     0|   0|    0|       0|
|          0|       0|     0|   0|  0|  0|    0|    0|     0|   0|    1|       0|
|          0|       0|     0|   0|  0|  0|    0|    0|     0|   0|    0|       0|
|          0|       0|     0|   0|  0|  0|    0|    0|     0|   0|    1|       0|
|          0|       0|     0|   0|  0|  1|    0|    0|     0|   0|    1|       0|
|          0|       0|     0|   0|  0|  0|    0|    0|     0|   0|    0|       0|
|          0|       0|     0|   0|  0|  0|    0|    0|     0|   0|    1|    

In [ ]:
print('Train columns with null values:')
print('-'*25)
data_raw.select([eval('data_raw.' + x + '.isNull().cast("int").alias("' + x + '")') for x in data_raw.columns]).\
    groupBy().sum().toPandas()

Train columns with null values:
-------------------------


,sum(PassengerId),sum(Survived),sum(Pclass),sum(Name),sum(Sex),sum(Age),sum(SibSp),sum(Parch),sum(Ticket),sum(Fare),sum(Cabin),sum(Embarked)
0,0,0,0,0,0,177,0,0,0,0,687,2


In [ ]:
print('Test columns with null values:')
print('-'*25)
data_val.select([eval('data_val.' + x + '.isNull().cast("int").alias("' + x + '")') for x in data_val.columns]).\
    groupBy().sum().toPandas()

Test columns with null values:
-------------------------


,sum(PassengerId),sum(Pclass),sum(Name),sum(Sex),sum(Age),sum(SibSp),sum(Parch),sum(Ticket),sum(Fare),sum(Cabin),sum(Embarked)
0,0,0,0,0,86,0,0,0,1,327,0


#### Above stuffs are normally things we need to prepare before jumping into any further steps of learning

### Exercise
1. Check and fill NA for NULL position using Spark.
2. Create new cleaned train and test data after filling NA, correcting datatype.

### Exercise
Do EDA with Titanic dataset using Spark.

In [14]:
df = spark.read.csv('./train.csv', inferSchema=True, header=True)
df.show(5, truncate=False)
df.printSchema()
df.describe().toPandas()

+-----------+--------+------+---------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|Name                                               |Sex   |Age |SibSp|Parch|Ticket          |Fare   |Cabin|Embarked|
+-----------+--------+------+---------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
|1          |0       |3     |Braund, Mr. Owen Harris                            |male  |22.0|1    |0    |A/5 21171       |7.25   |NULL |S       |
|2          |1       |1     |Cumings, Mrs. John Bradley (Florence Briggs Thayer)|female|38.0|1    |0    |PC 17599        |71.2833|C85  |C       |
|3          |1       |3     |Heikkinen, Miss. Laina                             |female|26.0|0    |0    |STON/O2. 3101282|7.925  |NULL |S       |
|4          |1       |1     |Futrelle, Mrs. Jacques Heath (Lily May Peel)       |female|35.0|1    |0    |113803          |53

,summary,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,count,891,891,891,891,891,714,891,891,891,891,204,889
1,mean,446.0,0.3838383838383838,2.308641975308642,None,None,29.69911764705882,0.5230078563411896,0.38159371492704824,260318.54916792738,32.2042079685746,None,None
2,stddev,257.3538420152301,0.48659245426485753,0.8360712409770491,None,None,14.526497332334035,1.1027434322934315,0.8060572211299488,471609.26868834975,49.69342859718089,None,None
3,min,1,0,1,"""Andersson, Mr. August Edvard (""""Wennerstrom"""")""",female,0.42,0,0,110152,0.0,A10,C
4,max,891,1,3,"van Melkebeke, Mr. Philemon",male,80.0,8,6,WE/P 5735,512.3292,T,S


In [15]:
from pyspark.sql.functions import col, isnan, count, when
from pyspark.sql.types import DoubleType, FloatType

exprs = []

for field in df.schema.fields:
    c = field.name
    dtype = field.dataType

    # Numeric → check NULL + NaN
    if isinstance(dtype, (DoubleType, FloatType)):
        exprs.append(
            count(
                when(col(c).isNull() | isnan(col(c)), c)
            ).alias(c)
        )
    # Non-numeric → chỉ check NULL
    else:
        exprs.append(
            count(
                when(col(c).isNull(), c)
            ).alias(c)
        )

missing_df = df.select(exprs)
missing_df.show(truncate=False)

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|0          |0       |0     |0   |0  |177|0    |0    |0     |0   |687  |2       |
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



In [19]:
cols_to_be_fill = ['Age', 'Cabin', 'Embarked']
df.select(*cols_to_be_fill).describe().show()

+-------+------------------+-----+--------+
|summary|               Age|Cabin|Embarked|
+-------+------------------+-----+--------+
|  count|               714|  204|     889|
|   mean| 29.69911764705882| NULL|    NULL|
| stddev|14.526497332334035| NULL|    NULL|
|    min|              0.42|  A10|       C|
|    max|              80.0|    T|       S|
+-------+------------------+-----+--------+



In [20]:
age_median = df.approxQuantile("Age", [0.5], 0.01)[0]
df = df.fillna({"Age": age_median})


In [26]:
df = df.fillna({"Cabin": "Unknown"})


In [22]:
from pyspark.sql.functions import col

embarked_mode = (
    df.groupBy("Embarked")
      .count()
      .orderBy(col("count").desc())
      .first()[0]
)

df = df.fillna({"Embarked": embarked_mode})


In [30]:
from pyspark.sql.functions import sum as spark_sum

df.select([
    spark_sum(col(c).isNull().cast("int")).alias(c)
    for c in cols_to_be_fill
]).show()


+---+-----+--------+
|Age|Cabin|Embarked|
+---+-----+--------+
|  0|    0|       0|
+---+-----+--------+



In [27]:
df.select(*cols_to_be_fill).describe().show()

+-------+------------------+-------+--------+
|summary|               Age|  Cabin|Embarked|
+-------+------------------+-------+--------+
|  count|               891|    891|     891|
|   mean| 29.36158249158249|   NULL|    NULL|
| stddev|13.019696550973201|   NULL|    NULL|
|    min|              0.42|    A10|       C|
|    max|              80.0|Unknown|       S|
+-------+------------------+-------+--------+



# Decision tree classification with pyspark

In [ ]:
cuse = spark.read.csv('./cuse_binary.csv', header=True, inferSchema=True)
cuse.show(5)

+---+---------+---------+---+
|age|education|wantsMore|  y|
+---+---------+---------+---+
|<25|      low|      yes|  0|
|<25|      low|      yes|  0|
|<25|      low|      yes|  0|
|<25|      low|      yes|  0|
|<25|      low|      yes|  0|
+---+---------+---------+---+
only showing top 5 rows



## Process categorical columns
The following code does three things with pipeline:

* **`StringIndexer`** all categorical columns
* **`OneHotEncoder`** all categorical index columns
* **`VectorAssembler`** all feature columns into one vector column

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline

# categorical columns
categorical_columns = cuse.columns[0:3]

#### Build StringIndexer stages

In [ ]:
stringindexer_stages = [StringIndexer(inputCol=c, outputCol='strindexed_' + c) for c in categorical_columns]
# encode label column and add it to stringindexer_stages
stringindexer_stages += [StringIndexer(inputCol='y', outputCol='label')]

#### Build OneHotEncoder stages

In [ ]:
onehotencoder_stages = [OneHotEncoder(inputCol='strindexed_' + c, outputCol='onehot_' + c) for c in categorical_columns]

#### Build VectorAssembler stage

In [ ]:
feature_columns = ['onehot_' + c for c in categorical_columns]
vectorassembler_stage = VectorAssembler(inputCols=feature_columns, outputCol='features')

#### Build pipeline model

In [ ]:
# all stages
all_stages = stringindexer_stages + onehotencoder_stages + [vectorassembler_stage]
pipeline = Pipeline(stages=all_stages)

In [ ]:
all_stages

[StringIndexer_e77108867df5,
 StringIndexer_49ded852e55f,
 StringIndexer_2020f01ea897,
 StringIndexer_f61ceb12a428,
 OneHotEncoder_cf47b323f01f,
 OneHotEncoder_9288694a5cdc,
 OneHotEncoder_8177dbe63ea9,
 VectorAssembler_d7873c96aec0]

#### Fit pipeline model

In [ ]:
pipeline_model = pipeline.fit(cuse)

#### Transform data

In [ ]:
final_columns = feature_columns + ['features', 'label']
cuse_df = pipeline_model.transform(cuse).\
            select(final_columns)

cuse_df.show(5)

+-------------+----------------+----------------+-------------------+-----+
|   onehot_age|onehot_education|onehot_wantsMore|           features|label|
+-------------+----------------+----------------+-------------------+-----+
|(3,[2],[1.0])|       (1,[],[])|   (1,[0],[1.0])|(5,[2,4],[1.0,1.0])|  0.0|
|(3,[2],[1.0])|       (1,[],[])|   (1,[0],[1.0])|(5,[2,4],[1.0,1.0])|  0.0|
|(3,[2],[1.0])|       (1,[],[])|   (1,[0],[1.0])|(5,[2,4],[1.0,1.0])|  0.0|
|(3,[2],[1.0])|       (1,[],[])|   (1,[0],[1.0])|(5,[2,4],[1.0,1.0])|  0.0|
|(3,[2],[1.0])|       (1,[],[])|   (1,[0],[1.0])|(5,[2,4],[1.0,1.0])|  0.0|
+-------------+----------------+----------------+-------------------+-----+
only showing top 5 rows



### Split data into training and test datasets

In [ ]:
training, test = cuse_df.randomSplit([0.8, 0.2], seed=1234)

### Build cross-validation model

#### Estimator

In [ ]:
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier

dt = DecisionTreeClassifier(featuresCol='features', labelCol='label')

#### Parameter grid

In [ ]:
from pyspark.ml.tuning import ParamGridBuilder
param_grid = ParamGridBuilder().\
    addGrid(dt.maxDepth, [2,3,4,5]).\
    build()

#### Evaluator

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", metricName="areaUnderROC")

#### Build cross-validation model

In [ ]:
from pyspark.ml.tuning import CrossValidator
cv = CrossValidator(estimator=dt, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=4)

In [ ]:
type(cv)

pyspark.ml.tuning.CrossValidator

#### Fit cross-validation mode

In [ ]:
cv_model = cv.fit(cuse_df)

#### Prediction

In [ ]:
show_columns = ['features', 'label', 'prediction', 'rawPrediction', 'probability']

##### Prediction on training data

In [ ]:
pred_training_cv = cv_model.transform(training)
pred_training_cv.select(show_columns).show(5, truncate=False)

+---------+-----+----------+-------------+----------------------------------------+
|features |label|prediction|rawPrediction|probability                             |
+---------+-----+----------+-------------+----------------------------------------+
|(5,[],[])|0.0  |1.0       |[203.0,237.0]|[0.46136363636363636,0.5386363636363637]|
|(5,[],[])|0.0  |1.0       |[203.0,237.0]|[0.46136363636363636,0.5386363636363637]|
|(5,[],[])|0.0  |1.0       |[203.0,237.0]|[0.46136363636363636,0.5386363636363637]|
|(5,[],[])|0.0  |1.0       |[203.0,237.0]|[0.46136363636363636,0.5386363636363637]|
|(5,[],[])|0.0  |1.0       |[203.0,237.0]|[0.46136363636363636,0.5386363636363637]|
+---------+-----+----------+-------------+----------------------------------------+
only showing top 5 rows



##### Prediction on test data

In [ ]:
pred_test_cv = cv_model.transform(test)
pred_test_cv.select(show_columns).show(5, truncate=False)

+---------+-----+----------+-------------+----------------------------------------+
|features |label|prediction|rawPrediction|probability                             |
+---------+-----+----------+-------------+----------------------------------------+
|(5,[],[])|0.0  |1.0       |[203.0,237.0]|[0.46136363636363636,0.5386363636363637]|
|(5,[],[])|0.0  |1.0       |[203.0,237.0]|[0.46136363636363636,0.5386363636363637]|
|(5,[],[])|0.0  |1.0       |[203.0,237.0]|[0.46136363636363636,0.5386363636363637]|
|(5,[],[])|0.0  |1.0       |[203.0,237.0]|[0.46136363636363636,0.5386363636363637]|
|(5,[],[])|0.0  |1.0       |[203.0,237.0]|[0.46136363636363636,0.5386363636363637]|
+---------+-----+----------+-------------+----------------------------------------+
only showing top 5 rows



### Confusion matrix

Pyspark doesn’t have a function to calculate the confusion matrix automatically, but we can still easily get a confusion matrix with a combination use of several methods from the RDD class.

In [ ]:
label_and_pred = cv_model.transform(cuse_df).select('label', 'prediction')
label_and_pred.rdd.zipWithIndex().countByKey()

defaultdict(int,
            {Row(label=0.0, prediction=0.0): 897,
             Row(label=0.0, prediction=1.0): 203,
             Row(label=1.0, prediction=0.0): 270,
             Row(label=1.0, prediction=1.0): 237})

### Parameters from the best model

In [ ]:
print('The best MaxDepth is:', cv_model.bestModel._java_obj.getMaxDepth())

The best MaxDepth is: 4


# Gradient boost tree classification

In [ ]:
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

gbt = GBTClassifier(featuresCol='features', labelCol='label')

param_grid = ParamGridBuilder().\
    addGrid(gbt.maxDepth, [2, 3, 4]).\
    addGrid(gbt.minInfoGain, [0.0, 0.1, 0.2, 0.3]).\
    addGrid(gbt.stepSize, [0.05, 0.1, 0.2, 0.4]).\
    build()

evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction')

In [ ]:
from pyspark.ml.tuning import CrossValidator

crossvalidation = CrossValidator(estimator=gbt, estimatorParamMaps=param_grid, evaluator=evaluator)

In [ ]:
crossvalidation_mod = crossvalidation.fit(cuse_df)

In [ ]:
# In train
pred_train = crossvalidation_mod.transform(training)
pred_train.show(5)

+----------+----------------+----------------+---------+-----+--------------------+--------------------+----------+
|onehot_age|onehot_education|onehot_wantsMore| features|label|       rawPrediction|         probability|prediction|
+----------+----------------+----------------+---------+-----+--------------------+--------------------+----------+
| (3,[],[])|       (1,[],[])|       (1,[],[])|(5,[],[])|  0.0|[-0.0345409316074...|[0.48273639924591...|       1.0|
| (3,[],[])|       (1,[],[])|       (1,[],[])|(5,[],[])|  0.0|[-0.0345409316074...|[0.48273639924591...|       1.0|
| (3,[],[])|       (1,[],[])|       (1,[],[])|(5,[],[])|  0.0|[-0.0345409316074...|[0.48273639924591...|       1.0|
| (3,[],[])|       (1,[],[])|       (1,[],[])|(5,[],[])|  0.0|[-0.0345409316074...|[0.48273639924591...|       1.0|
| (3,[],[])|       (1,[],[])|       (1,[],[])|(5,[],[])|  0.0|[-0.0345409316074...|[0.48273639924591...|       1.0|
+----------+----------------+----------------+---------+-----+----------

In [ ]:
# In test
pred_test = crossvalidation_mod.transform(test)
pred_test.show(5)

+----------+----------------+----------------+---------+-----+--------------------+--------------------+----------+
|onehot_age|onehot_education|onehot_wantsMore| features|label|       rawPrediction|         probability|prediction|
+----------+----------------+----------------+---------+-----+--------------------+--------------------+----------+
| (3,[],[])|       (1,[],[])|       (1,[],[])|(5,[],[])|  0.0|[-0.0345409316074...|[0.48273639924591...|       1.0|
| (3,[],[])|       (1,[],[])|       (1,[],[])|(5,[],[])|  0.0|[-0.0345409316074...|[0.48273639924591...|       1.0|
| (3,[],[])|       (1,[],[])|       (1,[],[])|(5,[],[])|  0.0|[-0.0345409316074...|[0.48273639924591...|       1.0|
| (3,[],[])|       (1,[],[])|       (1,[],[])|(5,[],[])|  0.0|[-0.0345409316074...|[0.48273639924591...|       1.0|
| (3,[],[])|       (1,[],[])|       (1,[],[])|(5,[],[])|  0.0|[-0.0345409316074...|[0.48273639924591...|       1.0|
+----------+----------------+----------------+---------+-----+----------

In [ ]:
# Performance
print('Accuracy on training data (areaUnderROC): ', evaluator.setMetricName('areaUnderROC').evaluate(pred_train), "\n"
      'Accuracy on test data (areaUnderROC): ', evaluator.setMetricName('areaUnderROC').evaluate(pred_test))


Accuracy on training data (areaUnderROC):  0.632389174374786 
Accuracy on test data (areaUnderROC):  0.6811111111111111


In [ ]:
# Confusion matrix in train
label_pred_train = pred_train.select('label', 'prediction')
label_pred_train.rdd.zipWithIndex().countByKey()

defaultdict(int,
            {Row(label=0.0, prediction=0.0): 708,
             Row(label=0.0, prediction=1.0): 167,
             Row(label=1.0, prediction=0.0): 227,
             Row(label=1.0, prediction=1.0): 190})

In [ ]:
# Confusion matrix in test
label_pred_test = pred_test.select('label', 'prediction')
label_pred_test.rdd.zipWithIndex().countByKey()

defaultdict(int,
            {Row(label=0.0, prediction=0.0): 189,
             Row(label=0.0, prediction=1.0): 36,
             Row(label=1.0, prediction=0.0): 43,
             Row(label=1.0, prediction=1.0): 47})

In [ ]:
# Best model and params
print('max depth: ', crossvalidation_mod.bestModel._java_obj.getMaxDepth(), "\n",
     'min information gain: ', crossvalidation_mod.bestModel._java_obj.getMinInfoGain(), "\n",
     'step size: ', crossvalidation_mod.bestModel._java_obj.getStepSize())

max depth:  2 
 min information gain:  0.0 
 step size:  0.2


### Exercise:
Do the classification in the case of cuse binary dataset above for other ML model in Spark.

```python
from pyspark.ml.classification import LinearSVC
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.classification import OneVsRest
```

## Exercise
Build decision tree and Gradient boosting classifier for the data Titanic above.


### Exercise

Do the classification in the case of data Titanic for other ML model in Spark.
```python
from pyspark.ml.classification import LinearSVC
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.classification import OneVsRest
```

### The other approaches will be taken as exercises